In [1]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections

# NLTK for NLP utils and corpora
import nltk

# NumPy and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1."))

# Helper libraries
from common import utils, vocabulary, tf_embed_viz, treeviz
from common import patched_numpy_io

In [2]:
bias_sentences = pd.read_csv("bias_sentences.csv")

In [3]:
bias_sentences.count()

sentence    113299
label       113299
dtype: int64

In [4]:
def weak_annotator(sent, list1, list2):
    ind1 = ind2 = len(sent)
    if any(word in sent for word in list1):
        ind1 = max([sent.lower().find(word) for word in list1])
    elif any(word in sent for word in list2):
        ind2 = max([sent.lower().find(word) for word in list2])
    else:
        ind1 = ind2 = len(sent)
        
    if ind1 < ind2:
        return 'liberal'
    elif ind2 < ind1:
        return 'conservative'
    else: 
        return 'neutral'       

In [5]:
lib_list = ['abortion',
 'congress',
 'rights',
 'anti',
 'law',
 'court',
 'party',
 'government',
 'women',
 'federal',
 'caucuses',
 'issue',
 'national',
 'right',
 'people',
 'support',
 'million',
 'health',
 'public',
 'city',
 'make',
 '000',
 'caucus',
 'political',
 'percent',
 'supreme',
 'like',
 'groups',
 'time',
 'money',
 'legislature',
 'say',
 'decision',
 'civil',
 'american',
 'life',
 'mr',
 'bush']

In [6]:
con_list = ['house',
 'committee',
 'senator',
 'leader',
 'democrat',
 'approved',
 'majority',
 'chairman',
 'republicans',
 'democratic',
 'democrats',
 'week',
 'budget',
 'measure',
 'assembly',
 'legislation',
 'members',
 'nomination',
 'united',
 'expected',
 'floor',
 'white',
 'campaign',
 'hearings',
 'judiciary',
 'plan',
 'voted',
 'leaders',
 'finance',
 'judge',
 'banking',
 'reagan',
 'governor',
 'conference',
 'intelligence',
 'dole',
 'senate',
 'republican',
 'today',
 'president',
 'new',
 'year',
 'passed',
 'states',
 'administration',
 'tax',
 'billion',
 'years']


In [7]:
ibc_database = pd.read_csv("ibcData.csv",sep='\t', header=None)

In [8]:
ibc_database.columns

Int64Index([0, 1], dtype='int64')

In [9]:
ibc_database[1].unique()

array(['liberal', 'conservative', 'neutral'], dtype=object)

In [10]:
weak_labels_ibc = []
 
for i in range(ibc_database.count()[0]):
    weak_labels_ibc.append(weak_annotator(ibc_database.iloc[i][0],lib_list,con_list))

In [11]:
weak_labels_bias = []

for i in range(bias_sentences.sentence.count()):
    weak_labels_bias.append(weak_annotator(bias_sentences.sentence[i],lib_list,con_list))

In [12]:
#bias_sentences = pd.DataFrame(data = nyt_reddit,columns = ['sentence'])
bias_sentences['label'] = weak_labels_bias

In [13]:
bias_sentences.head()

,sentence,label
0,"In the last 365 days, we've had problems: crit...",liberal
1,"As expected, the end of the quarter and the ye...",conservative
2,This traditional ''window-dressing'' helped sh...,liberal
3,Last year I was spot-on about the collapse of ...,conservative
4,THE average consumer's telephone bill has ris...,liberal


In [14]:
weak_labels_ibc.count('liberal')

2847

In [15]:
weak_labels_ibc.count('neutral')

1016

In [16]:
weak_labels_ibc.count('conservative')

463

In [17]:
ibc_database['weak_label'] = weak_labels_ibc

In [18]:
ibc_database.head()

,0,1,weak_label
0,Forcing middle-class workers to bear a greater...,liberal,liberal
1,Because it would not be worthwhile to bring a ...,liberal,neutral
2,"Indeed , Lind argues that high profits and hig...",liberal,neutral
3,"In fairness , it should be noted that he devot...",liberal,liberal
4,Psychological tactics are social control techn...,liberal,neutral


In [19]:
# tokenize IBC database
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

X = ibc_database[0]
U = bias_sentences['sentence']

X_tokens = []
U_tokens = []

for i in range(len(X)):
    X_tokens.append(tokenizer.tokenize(X[i]))
    
for i in range(len(U)):
    U_tokens.append(tokenizer.tokenize(U[i]))

In [20]:
# canonicalize IBC database

X_tokens_canon = []
U_tokens_canon = []

for i in range(len(X_tokens)):
    X_tokens_canon.append(utils.canonicalize_words(X_tokens[i]))
    
for i in range(len(U_tokens)):
    U_tokens_canon.append(utils.canonicalize_words(U_tokens[i]))

In [21]:
# flatten the list of lists consisting of canonicalized sentences

X_final = list(itertools.chain.from_iterable(X_tokens_canon))
U_final = list(itertools.chain.from_iterable(U_tokens_canon))

In [22]:
# number of unique tokens in database
print(len(set(X_final)))
print(len(set(U_final)))

15721
54687


In [23]:
# create unigrams, could be used for creating distribution plots

unigram_counts_X = collections.Counter()
unigram_counts_U = collections.Counter()

for word in X_final:  
    unigram_counts_X[word] += 1
    
for word in U_final:  
    unigram_counts_U[word] += 1

In [24]:
words_X = unigram_counts_X.keys()

counts_X = []
for w in words_X:
    counts_X.append(unigram_counts_X[w])
    
words_U = unigram_counts_U.keys()

counts_U = []
for w in words_U:
    counts_U.append(unigram_counts_U[w])

In [25]:
# sorting the words according to their counts

unigram_list_X = [x for _,x in sorted(zip(counts_X,words_X),  reverse=True)]
unigram_count_X = [y for y,_ in sorted(zip(counts_X,words_X),  reverse=True)]

unigram_list_U = [x for _,x in sorted(zip(counts_U,words_U),  reverse=True)]
unigram_count_U = [y for y,_ in sorted(zip(counts_U,words_U),  reverse=True)]

In [26]:
# plot the distribution of unigrams

utils.require_package("bokeh")
import bokeh.plotting as bp
from bokeh.models import HoverTool
bp.output_notebook()

Loading BokehJS ...

In [27]:
hist, bin_edges = np.histogram(a=unigram_count_X, bins=40, normed=True)

In [28]:
nplot = 100
fig = bp.figure(x_range=unigram_list_X[:nplot], plot_width=1000, plot_height=600)
bars = fig.vbar(x=unigram_list_X[:nplot], width=0.4, top=unigram_count_X[:nplot], hover_fill_color="firebrick")
fig.add_tools(HoverTool(tooltips=[("word", "@x"), ("count", "@top")], renderers=[bars], mode="vline"))
fig.y_range.start = 0
fig.y_range.end = 1.2*max(unigram_count_X)
fig.yaxis.axis_label = "Count(w)"
fig.xgrid.grid_line_alpha = 0.75
fig.xaxis.major_label_orientation = "vertical"
bp.show(fig)

In [29]:
hist, bin_edges = np.histogram(a=unigram_count_U, bins=40, normed=True)

In [30]:
nplot = 100
fig = bp.figure(x_range=unigram_list_U[:nplot], plot_width=1000, plot_height=600)
bars = fig.vbar(x=unigram_list_U[:nplot], width=0.4, top=unigram_count_U[:nplot], hover_fill_color="firebrick")
fig.add_tools(HoverTool(tooltips=[("word", "@x"), ("count", "@top")], renderers=[bars], mode="vline"))
fig.y_range.start = 0
fig.y_range.end = 1.2*max(unigram_count_U)
fig.yaxis.axis_label = "Count(w)"
fig.xgrid.grid_line_alpha = 0.75
fig.xaxis.major_label_orientation = "vertical"
bp.show(fig)

In [31]:
# initially we set vocabulary size equal to total number of unique tokens in the two dataset

mergelist = set(unigram_list_X + unigram_list_U)

V = len(mergelist)

mergelist_XU = X_final + U_final


vocab = vocabulary.Vocabulary(mergelist_XU, size=V)

In [32]:
# find the maximum length of sentences
lengths_X = map(len,X_tokens_canon)
lengths_U = map(len,U_tokens_canon)

In [33]:
lengths_X = list(lengths_X)
lengths_U = list(lengths_U)
max_len1 = max(lengths_X)
max_len2 = max(lengths_U)
max_len = max(max_len1,max_len2)

In [34]:
X_ids = []
U_ids = []
for i in range(len(X_tokens_canon)):
    X_ids.append(vocab.words_to_ids(X_tokens_canon[i]))
    
for i in range(len(U_tokens_canon)):
    U_ids.append(vocab.words_to_ids(U_tokens_canon[i]))

In [35]:
# final padded sentences
X_sent, ns = utils.pad_np_array(X_ids, max_len=max_len1, pad_id=0)
U_sent, u_ns = utils.pad_np_array(U_ids, max_len=max_len2, pad_id=0)

In [36]:
print(X_sent.shape)
print(U_sent.shape)

(4326, 86)
(113299, 493)


In [37]:
Y = ibc_database[1]
Y_u = bias_sentences['label']

In [38]:
Y = list(Y)
Y_u = list(Y_u)

In [39]:
# final labels
Y_label = []

for i in range(len(Y)):
    if Y[i] == 'liberal':
        Y_label.append(0)
    elif Y[i] == 'neutral':
        Y_label.append(1)
    else:
        Y_label.append(2)

Y_u_label = []

for i in range(len(Y_u)):
    if Y_u[i] == 'liberal':
        Y_u_label.append(0)
    elif Y_u[i] == 'neutral':
        Y_u_label.append(1)
    else:
        Y_u_label.append(2)

In [40]:
Y_w = ibc_database['weak_label']

Y_w =list(Y_w)
Y_weak = []
for i in range(len(Y)):
    if Y_w[i] == 'liberal':
        Y_weak.append(0)
    elif Y_w[i] == 'neutral':
        Y_weak.append(1)
    else:
        Y_weak.append(2)

In [41]:
X_sent_w = []
for i in range(X_sent.shape[0]):
    X_sent_w.append(np.append(X_sent[i], [Y_weak[i],ns[i]]))

U_sent_w = []
for i in range(U_sent.shape[0]):
    U_sent_w.append(np.append(U_sent[i], [u_ns[i]]))

In [42]:
X_sent_w = np.reshape(X_sent_w, newshape= (X_sent.shape[0],X_sent.shape[1]+2))
U_sent_w = np.reshape(U_sent_w, newshape= (U_sent.shape[0],U_sent.shape[1]+1))

In [43]:
len(Y_u_label)

113299

In [44]:
from sklearn.model_selection import train_test_split 

X_train,X_test,y_train, y_test = train_test_split(X_sent_w, Y_label, test_size=0.2, random_state=42)
U_train,U_test,y_u_train, y_u_test = train_test_split(U_sent_w, Y_u_label, test_size=0.2, random_state=42)

In [45]:
ns_train = []
ns_test = []

ns_u_train = []
ns_u_test = []

for i in range(X_train.shape[0]):
    ns_train.append(X_train[i][-1])
for i in range(X_test.shape[0]):
    ns_test.append(X_test[i][-1])
    
for i in range(U_train.shape[0]):
    ns_u_train.append(U_train[i][-1])
for i in range(U_test.shape[0]):
    ns_u_test.append(U_test[i][-1])

In [46]:
X_n_train = np.delete(X_train,len(X_train[1])-1,axis=1)
X_n_test  = np.delete(X_test,len(X_test[1])-1,axis=1)

U_n_train = np.delete(U_train,len(U_train[1])-1,axis=1)
U_n_test  = np.delete(U_test,len(U_test[1])-1,axis=1)

In [47]:
# Now we define our model
from __future__ import print_function
from __future__ import division

import tensorflow as tf

def embedding_layer(ids_, V, embed_dim, init_scale=0.001):
    
    W_embed_ = tf.get_variable(name = 'W_embed', shape=[V,embed_dim],dtype=tf.float32
                               ,initializer=tf.random_uniform_initializer(minval= -init_scale
                                                                          ,maxval =init_scale),trainable=True)
    xs_ = tf.nn.embedding_lookup(W_embed_, ids_)
    return xs_

In [48]:
def fully_connected_layers(h0_, hidden_dims, activation=tf.tanh,
                           dropout_rate=0, is_training=False):
    h_ = h0_
    for i, hdim in enumerate(hidden_dims):
        
        h_ = tf.layers.dense(h_, hdim, activation=activation,    
                             kernel_initializer=tf.contrib.layers.xavier_initializer()
                             , bias_initializer=tf.zeros_initializer(), name = 'Hidden_%d'%i)
        
        if dropout_rate > 0:
            h_ = tf.nn.dropout(h_, keep_prob=dropout_rate)
            
        #if dropout_rate > 0:
            #h_ = tf.layers.dropout(inputs = h_, rate=dropout_rate, training = is_training)
        
    return h_

In [49]:
def softmax_output_layer(h_, labels_, num_classes):
    
    n = h_.get_shape().as_list()
    
    with tf.variable_scope("Logits"):
        W_out_ = tf.get_variable(name = 'W_out', shape=[n[1],num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        b_out_ = tf.get_variable(name = 'b_out', shape=[num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        logits_ = tf.matmul(h_,W_out_) + b_out_
        
        

    # If no labels provided, don't try to compute loss.
    if labels_ is None:
        return None, logits_

    with tf.name_scope("Softmax"):
        loss_ = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_,logits=logits_)) 
        
    
    return loss_, logits_

In [50]:
def BOW_encoder(ids_, ns_, V, embed_dim, hidden_dims, dropout_rate=0,is_training=None,
                **unused_kw):
    
    assert is_training is not None, "is_training must be explicitly set to True or False"
    
    with tf.variable_scope("Embedding_Layer"):
        xs_ = embedding_layer(ids_, V, embed_dim, init_scale=0.001) 
    mask_ = tf.expand_dims(tf.sequence_mask(ns_, xs_.shape[1],dtype=tf.float32), -1)
    xs_ =  mask_ * xs_
    x_ = tf.reduce_sum(xs_,axis = 1)
    h_ = fully_connected_layers(x_, hidden_dims, activation=tf.tanh, dropout_rate=dropout_rate,is_training=is_training)   
        
    return h_, xs_

In [51]:
def classifier_model_fn(features, labels, mode, params):
    # Seed the RNG for repeatability
    tf.set_random_seed(params.get('rseed', 10))

    # Check if this graph is going to be used for training.
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    if params['encoder_type'] == 'bow':
        with tf.variable_scope("Encoder"):
            h_, xs_ = BOW_encoder(features['ids'], features['ns'],
                                  is_training=is_training,
                                  **params)
    else:
        raise ValueError("Error: unsupported encoder type "
                         "'{:s}'".format(params['encoder_type']))

    # Construct softmax layer and loss functions
    with tf.variable_scope("Output_Layer"):
        ce_loss_, logits_ = softmax_output_layer(h_, labels, params['num_classes'])

    with tf.name_scope("Prediction"):
        pred_proba_ = tf.nn.softmax(logits_, name="pred_proba")
        pred_max_ = tf.argmax(logits_, 1, name="pred_max")
        predictions_dict = {"proba": pred_proba_, "max": pred_max_}

    if mode == tf.estimator.ModeKeys.PREDICT:
        # If predict mode, don't bother computing loss
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions_dict)

    # L2 regularization (weight decay) on parameters, from all layers
    with tf.variable_scope("Regularization"):
        l2_penalty_ = tf.nn.l2_loss(xs_)  # l2 loss on embeddings
        for var_ in tf.trainable_variables():
            if "Embedding_Layer" in var_.name:
                continue
            l2_penalty_ += tf.nn.l2_loss(var_)
        l2_penalty_ *= params['beta']  # scale by regularization strength
        tf.summary.scalar("l2_penalty", l2_penalty_)
        regularized_loss_ = ce_loss_ + l2_penalty_

    with tf.variable_scope("Training"):
        if params['optimizer'] == 'adagrad':
            optimizer_ = tf.train.AdagradOptimizer(params['lr'])
        else:
            optimizer_ = tf.train.GradientDescentOptimizer(params['lr'])
        train_op_ = optimizer_.minimize(regularized_loss_,
                                        global_step=tf.train.get_global_step())

    tf.summary.scalar("cross_entropy_loss", ce_loss_)
    eval_metrics = {"cross_entropy_loss": tf.metrics.mean(ce_loss_),
                    "accuracy": tf.metrics.accuracy(labels, pred_max_)}

    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=predictions_dict,
                                      loss=regularized_loss_,
                                      train_op=train_op_,
                                      eval_metric_ops=eval_metrics)

In [52]:
# training the model (full supervision)

batch_size = 5
e = 175 # embedding dimension
h = [32,64,128] # hidden dimensions
l = 0.001 # learning rate
num_epoch = 10 #no. of epochs

# Specify model hyperparameters as used by model_fn
model_params = dict(V=V, embed_dim=e, hidden_dims=h, num_classes=3,encoder_type='bow',lr=l,optimizer='adagrad'
                    , beta=0.001)
model_fn = classifier_model_fn


with tf.Graph().as_default(), tf.Session() as sess:
    
    x_ph_  = tf.placeholder(tf.int32, shape=[None, X_n_train.shape[1]])  # [batch_size, max_len]
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    y_ph_  = tf.placeholder(tf.int32, shape=[None])              # [batch_size]
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.TRAIN,
                              params=model_params)
    loss_     = estimator_spec.loss
    train_op_ = estimator_spec.train_op
    
    
    sess.run(tf.global_variables_initializer())
    
    total_loss = 0
    loss_ema = np.log(2)  # track exponential-moving-average of loss
    ema_decay = np.exp(-1/10)  # decay parameter for moving average = np.exp(-1/history_length)
    
    
    y_conf = []
     
    for i in range(len(y_train)):
        y_conf.append(2 - np.absolute(Y_weak[i]-y_train[i]))
       
           
    
    y_conf = np.reshape(y_conf,newshape=(len(y_conf),))
    ns_train = np.reshape(ns_train,newshape=(len(ns_train),))
    
    for j in range(num_epoch):
        
        t0 = time.time()
        total_batches = 0
        total_examples = 0
        
        for (bx, bns, by) in utils.multi_batch_generator(batch_size, X_n_train, ns_train, y_conf):
            # feed NumPy arrays into the placeholder Tensors
            feed_dict = {x_ph_: bx, ns_ph_: bns, y_ph_: by}
            batch_loss, _ = sess.run([loss_, train_op_], feed_dict=feed_dict)
        
            # Compute some statistics
            total_batches += 1
            total_examples += len(bx)
            total_loss += batch_loss * len(bx)  # re-scale, since batch loss is mean
            # Compute moving average to smooth out noisy per-batch loss
            loss_ema = ema_decay * loss_ema + (1 - ema_decay) * batch_loss
        
            if (total_batches % 100 == 0):
                print("{:5,} examples, moving-average loss {:.2f}".format(total_examples, 
                                                                      loss_ema))    
        print("Completed no. {:d} epoch in {:s}".format(j+1,utils.pretty_timedelta(since=t0)))


  500 examples, moving-average loss 1.41
1,000 examples, moving-average loss 1.43
1,500 examples, moving-average loss 1.33
2,000 examples, moving-average loss 1.38
2,500 examples, moving-average loss 1.43
3,000 examples, moving-average loss 1.40
Completed no. 1 epoch in 0:00:05
  500 examples, moving-average loss 1.03
1,000 examples, moving-average loss 1.06
1,500 examples, moving-average loss 1.02
2,000 examples, moving-average loss 1.04
2,500 examples, moving-average loss 1.22
3,000 examples, moving-average loss 1.16
Completed no. 2 epoch in 0:00:02
  500 examples, moving-average loss 0.79
1,000 examples, moving-average loss 0.77
1,500 examples, moving-average loss 0.75
2,000 examples, moving-average loss 0.71
2,500 examples, moving-average loss 0.89
3,000 examples, moving-average loss 0.78
Completed no. 3 epoch in 0:00:02
  500 examples, moving-average loss 0.59
1,000 examples, moving-average loss 0.56
1,500 examples, moving-average loss 0.53
2,000 examples, moving-average loss 0.52

In [54]:
with tf.Graph().as_default(), tf.device('/gpu:0'),tf.Session() as sess:
    
    x_ph_  = tf.placeholder(tf.int32, shape=[None, X_n_test.shape[1]])  
    ns_ph_ = tf.placeholder(tf.int32, shape=[None])              
    y_ph_  = tf.placeholder(tf.int32, shape=[None])             
    
    # Construct the graph using model_fn
    features = {"ids": x_ph_, "ns": ns_ph_}  # note that values are Tensors
    estimator_spec = model_fn(features, labels=y_ph_, mode=tf.estimator.ModeKeys.PREDICT,
                              params=model_params)
    predict_ = estimator_spec.predictions
    
    sess.run(tf.global_variables_initializer())
    
    t = X_n_train.shape[0]
    
    true_conf_score = []
    for i in range(X_n_test.shape[0]):
        true_conf_score.append(2 - np.absolute(Y_weak[t+i]-y_test[i]))
    
    
    feed_dict = {x_ph_: X_n_test, ns_ph_: ns_test, y_ph_: true_conf_score}
    predictions = sess.run(predict_, feed_dict=feed_dict)

In [55]:
true_conf_score.count(0)

234

In [56]:
pred_conf_score = predictions['max']
len(true_conf_score)

866

In [57]:
conf_prob = predictions['proba']

t = X_n_train.shape[0]

for i in range(X_n_test.shape[0]):
    print(conf_prob[i]," test_label:",y_test[i]," weak_label:",Y_weak[t+i])

[ 0.07322557  0.03891807  0.88785636]  test_label: 2  weak_label: 1
[ 0.07860599  0.04398205  0.87741202]  test_label: 0  weak_label: 0
[ 0.07628779  0.04670405  0.87700808]  test_label: 2  weak_label: 0
[ 0.07480229  0.04216186  0.8830359 ]  test_label: 2  weak_label: 0
[ 0.07665514  0.0467688   0.87657601]  test_label: 0  weak_label: 0
[ 0.07649496  0.04859744  0.87490761]  test_label: 2  weak_label: 0
[ 0.07275506  0.04085319  0.88639176]  test_label: 0  weak_label: 0
[ 0.07269223  0.04755015  0.87975764]  test_label: 0  weak_label: 2
[ 0.07792452  0.04726973  0.87480581]  test_label: 0  weak_label: 2
[ 0.07670923  0.04410905  0.87918162]  test_label: 0  weak_label: 1
[ 0.07708305  0.04954216  0.87337482]  test_label: 1  weak_label: 0
[ 0.07896527  0.04551755  0.87551713]  test_label: 0  weak_label: 0
[ 0.07713     0.04543407  0.87743598]  test_label: 0  weak_label: 0
[ 0.07888339  0.04638249  0.8747341 ]  test_label: 2  weak_label: 2
[ 0.08079293  0.04982736  0.8693797 ]  test_labe

[ 0.07389481  0.04384984  0.88225532]  test_label: 0  weak_label: 2
[ 0.07844873  0.04633005  0.87522125]  test_label: 0  weak_label: 0
[ 0.08005022  0.04443716  0.87551266]  test_label: 2  weak_label: 2
[ 0.07763632  0.04711861  0.87524509]  test_label: 2  weak_label: 0
[ 0.08000875  0.04721426  0.87277699]  test_label: 0  weak_label: 0
[ 0.08033216  0.04886681  0.87080109]  test_label: 0  weak_label: 0
[ 0.07923782  0.04746956  0.87329262]  test_label: 2  weak_label: 1
[ 0.07665247  0.04337799  0.87996954]  test_label: 2  weak_label: 1
[ 0.07532654  0.04494201  0.87973148]  test_label: 0  weak_label: 2
[ 0.07312514  0.04436945  0.88250548]  test_label: 0  weak_label: 1
[ 0.07737187  0.04375817  0.87886995]  test_label: 1  weak_label: 0
[ 0.07863861  0.05245231  0.86890906]  test_label: 2  weak_label: 0
[ 0.07284531  0.04069449  0.88646019]  test_label: 1  weak_label: 1
[ 0.07571222  0.0425666   0.8817212 ]  test_label: 0  weak_label: 1
[ 0.07756718  0.04525257  0.87718022]  test_labe

[ 0.07669669  0.04692104  0.87638223]  test_label: 2  weak_label: 0
[ 0.07815537  0.05087761  0.87096697]  test_label: 0  weak_label: 1
[ 0.07682319  0.04480662  0.87837023]  test_label: 2  weak_label: 1
[ 0.07634269  0.0447303   0.87892705]  test_label: 0  weak_label: 2
[ 0.0778381   0.04818557  0.87397629]  test_label: 2  weak_label: 1
[ 0.07608122  0.04611478  0.87780398]  test_label: 2  weak_label: 0
[ 0.07718474  0.05229888  0.87051642]  test_label: 0  weak_label: 1
[ 0.07321989  0.05050107  0.87627906]  test_label: 2  weak_label: 0
[ 0.07773992  0.04859049  0.87366962]  test_label: 2  weak_label: 1
[ 0.07490274  0.0471005   0.8779968 ]  test_label: 1  weak_label: 0
[ 0.07589426  0.04836215  0.87574363]  test_label: 0  weak_label: 0
[ 0.07901502  0.04669275  0.87429219]  test_label: 1  weak_label: 2
[ 0.07132446  0.04332893  0.88534665]  test_label: 0  weak_label: 0
[ 0.07907503  0.0502668   0.87065822]  test_label: 0  weak_label: 1
[ 0.07902095  0.0480226   0.87295645]  test_labe

[ 0.07710686  0.0466591   0.87623399]  test_label: 0  weak_label: 1
[ 0.08089908  0.04917818  0.86992276]  test_label: 2  weak_label: 1
[ 0.0791147   0.04636082  0.87452441]  test_label: 0  weak_label: 2
[ 0.07971541  0.04585975  0.87442487]  test_label: 0  weak_label: 0
[ 0.07412148  0.04320603  0.88267249]  test_label: 2  weak_label: 0
[ 0.07702614  0.04719362  0.87578028]  test_label: 0  weak_label: 0
[ 0.07913478  0.04810462  0.87276059]  test_label: 2  weak_label: 0
[ 0.07382566  0.04688043  0.87929392]  test_label: 0  weak_label: 0
[ 0.07904511  0.04609101  0.87486392]  test_label: 2  weak_label: 2
[ 0.07859848  0.04807043  0.87333107]  test_label: 2  weak_label: 0
[ 0.07644848  0.04581723  0.87773424]  test_label: 0  weak_label: 0
[ 0.07695933  0.04704826  0.87599236]  test_label: 2  weak_label: 0
[ 0.0760684   0.04251555  0.88141608]  test_label: 2  weak_label: 0
[ 0.07700743  0.04364786  0.8793447 ]  test_label: 0  weak_label: 0
[ 0.07254839  0.04454376  0.88290781]  test_labe

[ 0.07458787  0.05035783  0.87505424]  test_label: 2  weak_label: 0
[ 0.07757503  0.04788876  0.87453622]  test_label: 0  weak_label: 1
[ 0.07562824  0.04926878  0.875103  ]  test_label: 1  weak_label: 0
[ 0.07831099  0.0459839   0.87570512]  test_label: 2  weak_label: 1
[ 0.07426885  0.04289051  0.88284063]  test_label: 0  weak_label: 1
[ 0.07805273  0.04921089  0.87273639]  test_label: 2  weak_label: 2
[ 0.07888001  0.04075365  0.88036638]  test_label: 2  weak_label: 0
[ 0.07635033  0.04661885  0.87703085]  test_label: 0  weak_label: 0
[ 0.08252972  0.04627773  0.87119251]  test_label: 0  weak_label: 0
[ 0.07785185  0.04717712  0.87497103]  test_label: 1  weak_label: 1
[ 0.07635922  0.04415928  0.87948149]  test_label: 2  weak_label: 1
[ 0.07771444  0.04954935  0.87273622]  test_label: 1  weak_label: 1
[ 0.07502719  0.04841599  0.87655675]  test_label: 0  weak_label: 0
[ 0.08142447  0.04641297  0.87216258]  test_label: 0  weak_label: 0
[ 0.07939268  0.04207359  0.87853372]  test_labe

In [58]:
num_correct = [pred_conf_score == true_conf_score]
sum(sum(num_correct))/len(pred_conf_score)

0.3625866050808314

In [59]:
model = tf.estimator.Estimator(model_fn=classifier_model_fn, 
                               params=model_params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tmpoo8qf2hh', '_save_summary_steps': 100, '_tf_random_seed': 1, '_keep_checkpoint_max': 5}


In [63]:
# training the model (full supervision) 2nd method
num_epoch = 50
train_params = dict(batch_size=30, total_epochs=num_epoch, eval_every=1)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.

ns_train = np.reshape(ns_train,newshape=(len(ns_train),))
y_conf = np.reshape(y_conf,newshape=(len(y_conf),))


train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": X_n_train, "ns": ns_train}, y=y_conf,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

ns_test = np.reshape(ns_test,newshape=(len(ns_test),))
true_conf_score = np.reshape(true_conf_score,newshape=(len(true_conf_score),))

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": X_n_test, "ns": ns_test}, y=true_conf_score,
                    batch_size=20, num_epochs=1, shuffle=False
                )

true_conf_score = np.reshape(true_conf_score,newshape=(len(true_conf_score),))

In [64]:
with tf.device('/gpu:0'):
    for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on test
        model.train(input_fn=train_input_fn)
        eval_metrics = model.evaluate(input_fn=test_input_fn, name="test")

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-2320
INFO:tensorflow:Saving checkpoints for 2321 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:loss = 0.307958, step = 2321
INFO:tensorflow:global_step/sec: 196.142
INFO:tensorflow:loss = 0.307248, step = 2421 (0.511 sec)
INFO:tensorflow:Saving checkpoints for 2436 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:Loss for final step: 0.290106.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:06:59
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-2436
INFO:tensorflow:Finished evaluation at 2018-04-16-20:06:59
INFO:tensorflow:Saving dict for global step 2436: accuracy = 0.364896, cross_entropy_loss = 2.60672, global_step = 2436, loss = 2.89108
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-2436
INFO:tensorflow:Saving checkpoints for 2437 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tens

INFO:tensorflow:loss = 0.293836, step = 3581 (0.519 sec)
INFO:tensorflow:Saving checkpoints for 3596 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:Loss for final step: 0.28587.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:07:22
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-3596
INFO:tensorflow:Finished evaluation at 2018-04-16-20:07:23
INFO:tensorflow:Saving dict for global step 3596: accuracy = 0.369515, cross_entropy_loss = 2.90766, global_step = 3596, loss = 3.19066
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-3596
INFO:tensorflow:Saving checkpoints for 3597 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:loss = 0.293889, step = 3597
INFO:tensorflow:global_step/sec: 193.657
INFO:tensorflow:loss = 0.293109, step = 3697 (0.517 sec)
INFO:tensorflow:Saving checkpoints for 3712 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:Loss for final step: 0.285585.
INFO:tensorflow:Starting 

INFO:tensorflow:Finished evaluation at 2018-04-16-20:07:47
INFO:tensorflow:Saving dict for global step 4756: accuracy = 0.371824, cross_entropy_loss = 3.07892, global_step = 4756, loss = 3.36033
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-4756
INFO:tensorflow:Saving checkpoints for 4757 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:loss = 0.288482, step = 4757
INFO:tensorflow:global_step/sec: 181.515
INFO:tensorflow:loss = 0.287981, step = 4857 (0.553 sec)
INFO:tensorflow:Saving checkpoints for 4872 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:Loss for final step: 0.283161.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:07:49
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-4872
INFO:tensorflow:Finished evaluation at 2018-04-16-20:07:49
INFO:tensorflow:Saving dict for global step 4872: accuracy = 0.371824, cross_entropy_loss = 3.09239, global_step = 4872, loss = 3.37364
INFO:tensor

INFO:tensorflow:Saving checkpoints for 5917 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:loss = 0.285012, step = 5917
INFO:tensorflow:global_step/sec: 177.444
INFO:tensorflow:loss = 0.28467, step = 6017 (0.565 sec)
INFO:tensorflow:Saving checkpoints for 6032 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:Loss for final step: 0.281087.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:08:13
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-6032
INFO:tensorflow:Finished evaluation at 2018-04-16-20:08:14
INFO:tensorflow:Saving dict for global step 6032: accuracy = 0.367206, cross_entropy_loss = 3.20435, global_step = 6032, loss = 3.48394
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-6032
INFO:tensorflow:Saving checkpoints for 6033 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:loss = 0.284717, step = 6033
INFO:tensorflow:global_step/sec: 177.124
INFO:tensorflow:loss = 0.284385, step = 613

INFO:tensorflow:Loss for final step: 0.279156.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:08:38
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-7192
INFO:tensorflow:Finished evaluation at 2018-04-16-20:08:39
INFO:tensorflow:Saving dict for global step 7192: accuracy = 0.37067, cross_entropy_loss = 3.28744, global_step = 7192, loss = 3.56534
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-7192
INFO:tensorflow:Saving checkpoints for 7193 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:loss = 0.282066, step = 7193
INFO:tensorflow:global_step/sec: 171.482
INFO:tensorflow:loss = 0.281805, step = 7293 (0.584 sec)
INFO:tensorflow:Saving checkpoints for 7308 into /tmp/tmpoo8qf2hh/model.ckpt.
INFO:tensorflow:Loss for final step: 0.278969.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:08:40
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-7308
INFO:tensorflow:Finished e

In [65]:
eval_metrics = model.evaluate(input_fn=test_input_fn, name="test")  # replace with result of model.evaluate(...)


print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

INFO:tensorflow:Starting evaluation at 2018-04-16-20:09:04
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-8120
INFO:tensorflow:Finished evaluation at 2018-04-16-20:09:04
INFO:tensorflow:Saving dict for global step 8120: accuracy = 0.369515, cross_entropy_loss = 3.3404, global_step = 8120, loss = 3.61695
Accuracy on test set: 36.95%


{'accuracy': 0.369515,
 'cross_entropy_loss': 3.3404,
 'global_step': 8120,
 'loss': 3.6169541}

In [66]:
def softmax_output_layer_conf(h_, labels_, num_classes, conf):
    
    n = h_.get_shape().as_list()
    
    with tf.variable_scope("Logits"):
        W_out_ = tf.get_variable(name = 'W_out', shape=[n[1],num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        b_out_ = tf.get_variable(name = 'b_out', shape=[num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        logits_ = tf.matmul(h_,W_out_) + b_out_
        
        

    # If no labels provided, don't try to compute loss.
    if labels_ is None:
        return None, logits_

    with tf.name_scope("Softmax"):
        #c = tf.constant(conf,dtype = tf.float32)
        loss_ = tf.reduce_mean(tf.cast(conf,dtype = tf.float32) * tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_,logits=logits_)) 
        
    
    return loss_, logits_

In [67]:
def classifier_model_fn_conf(features, labels, mode, params):
    # Seed the RNG for repeatability
    tf.set_random_seed(params.get('rseed', 10))

    # Check if this graph is going to be used for training.
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    if params['encoder_type'] == 'bow':
        with tf.variable_scope("Encoder"):
            h_, xs_ = BOW_encoder(features['ids'], features['ns'],
                                  is_training=is_training,
                                  **params)
    else:
        raise ValueError("Error: unsupported encoder type "
                         "'{:s}'".format(params['encoder_type']))

    # Construct softmax layer and loss functions
    with tf.variable_scope("Output_Layer"):
        ce_loss_, logits_ = softmax_output_layer_conf(h_, labels, params['num_classes'],conf=features['conf'])

    with tf.name_scope("Prediction"):
        pred_proba_ = tf.nn.softmax(logits_, name="pred_proba")
        pred_max_ = tf.argmax(logits_, 1, name="pred_max")
        predictions_dict = {"proba": pred_proba_, "max": pred_max_}

    if mode == tf.estimator.ModeKeys.PREDICT:
        # If predict mode, don't bother computing loss.
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions_dict)

    # L2 regularization (weight decay) on parameters, from all layers
    with tf.variable_scope("loss"):
        l2_penalty_ = tf.nn.l2_loss(xs_)  # l2 loss on embeddings
        for var_ in tf.trainable_variables():
            if "Embedding_Layer" in var_.name:
                continue
            l2_penalty_ += tf.nn.l2_loss(var_)
        l2_penalty_ *= params['beta']  # scale by regularization strength
        tf.summary.scalar("l2_penalty", l2_penalty_)
        regularized_loss_ = ce_loss_ + l2_penalty_
        

    with tf.variable_scope("Training"):
        if params['optimizer'] == 'adagrad':
            optimizer_ = tf.train.AdagradOptimizer(params['lr'])
        else:
            optimizer_ = tf.train.GradientDescentOptimizer(params['lr'])
        train_op_ = optimizer_.minimize(regularized_loss_,
                                        global_step=tf.train.get_global_step())

    tf.summary.scalar("cross_entropy_loss", ce_loss_)
    eval_metrics = {"cross_entropy_loss": tf.metrics.mean(ce_loss_),
                    "accuracy": tf.metrics.accuracy(labels, pred_max_)}

    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=predictions_dict,
                                      loss=regularized_loss_,
                                      train_op=train_op_,
                                      eval_metric_ops=eval_metrics)

In [68]:
true_conf_score = []
for i in range(U_n_train.shape[0]):
    true_conf_score.append(2)
    
ns_u_train = np.reshape(ns_u_train,newshape=(len(ns_u_train),))

train_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": U_n_train, "ns": ns_u_train},
                    batch_size=1000, num_epochs=1, shuffle=False
                )

predictions_train = model.predict(input_fn=train_input_fn)  

ns_u_test = np.reshape(ns_u_test,newshape=(len(ns_u_test),))

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": U_n_test, "ns": ns_u_test},
                    batch_size=1000, num_epochs=1, shuffle=False
                )

predictions_test = model.predict(input_fn=test_input_fn)

In [69]:
predictions_train =list(predictions_train)
predictions_test =list(predictions_test)

INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-8120
INFO:tensorflow:Restoring parameters from /tmp/tmpoo8qf2hh/model.ckpt-8120


In [70]:
conf_score_u_train = []

for i in range(len(predictions_train)):
    conf_score_u_train.append(predictions_train[i]['max'])
    
conf_score_u_test = []

for i in range(len(predictions_test)):
    conf_score_u_test.append(predictions_test[i]['max'])

In [71]:
model_params = dict(V=V, embed_dim=e, hidden_dims=h, num_classes=3,encoder_type='bow',lr=l,optimizer='adagrad'
                    , beta=0.001)


model1 = tf.estimator.Estimator(model_fn=classifier_model_fn_conf, 
                               params=model_params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tmpbzxc23gc', '_save_summary_steps': 100, '_tf_random_seed': 1, '_keep_checkpoint_max': 5}


In [72]:
# # training the model (weak supervision)
num_epochs = 200

train_params = dict(batch_size=250, total_epochs=num_epoch, eval_every=1)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

# Construct and train the model, saving checkpoints to the directory above.

ns_u_train = np.reshape(ns_u_train,newshape=(len(ns_u_train),))
y_u_train = np.reshape(y_u_train,newshape=(len(y_u_train),))
conf_score_u_train = np.reshape(conf_score_u_train,newshape=(U_n_train.shape[0],))

train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": U_n_train, "ns": ns_u_train,"conf": conf_score_u_train}, y=y_u_train,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

ns_u_test = np.reshape(ns_u_test,newshape=(len(ns_u_test),))
y_u_test = np.reshape(y_u_test,newshape=(len(y_u_test),))
conf_score_u_test = np.reshape(conf_score_u_test,newshape=(U_n_test.shape[0],))

test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": U_n_test, "ns": ns_u_test,"conf": conf_score_u_test}, y=y_u_test,
                    batch_size=1000, num_epochs=1, shuffle=False
                )

In [73]:
with tf.device('/device:GPU:0'):
    for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on test
        model1.train(input_fn=train_input_fn)
        eval_metrics = model1.evaluate(input_fn=test_input_fn, name="test")

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:loss = 1.81522, step = 1
INFO:tensorflow:global_step/sec: 12.0036
INFO:tensorflow:loss = 1.19107, step = 101 (8.332 sec)
INFO:tensorflow:global_step/sec: 12.0935
INFO:tensorflow:loss = 1.07852, step = 201 (8.269 sec)
INFO:tensorflow:global_step/sec: 12.0726
INFO:tensorflow:loss = 1.0551, step = 301 (8.283 sec)
INFO:tensorflow:Saving checkpoints for 363 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:Loss for final step: 0.75269.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:10:50
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-363
INFO:tensorflow:Finished evaluation at 2018-04-16-20:10:53
INFO:tensorflow:Saving dict for global step 363: accuracy = 0.838394, cross_entropy_loss = 0.603133, global_step = 363, loss = 0.894049
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc

INFO:tensorflow:loss = 0.416034, step = 3105 (7.294 sec)
INFO:tensorflow:global_step/sec: 13.7192
INFO:tensorflow:loss = 0.510902, step = 3205 (7.289 sec)
INFO:tensorflow:Saving checkpoints for 3267 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:Loss for final step: 0.378858.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:15:00
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-3267
INFO:tensorflow:Finished evaluation at 2018-04-16-20:15:03
INFO:tensorflow:Saving dict for global step 3267: accuracy = 0.931421, cross_entropy_loss = 0.271095, global_step = 3267, loss = 0.56988
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-3267
INFO:tensorflow:Saving checkpoints for 3268 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:loss = 0.427233, step = 3268
INFO:tensorflow:global_step/sec: 13.8902
INFO:tensorflow:loss = 0.409025, step = 3368 (7.201 sec)
INFO:tensorflow:global_step/sec: 13.7541
INFO:tens

INFO:tensorflow:Starting evaluation at 2018-04-16-20:19:04
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-6171
INFO:tensorflow:Finished evaluation at 2018-04-16-20:19:07
INFO:tensorflow:Saving dict for global step 6171: accuracy = 0.931377, cross_entropy_loss = 0.28631, global_step = 6171, loss = 0.582163
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-6171
INFO:tensorflow:Saving checkpoints for 6172 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:loss = 0.35006, step = 6172
INFO:tensorflow:global_step/sec: 13.8776
INFO:tensorflow:loss = 0.350653, step = 6272 (7.207 sec)
INFO:tensorflow:global_step/sec: 13.9248
INFO:tensorflow:loss = 0.334948, step = 6372 (7.181 sec)
INFO:tensorflow:global_step/sec: 13.9853
INFO:tensorflow:loss = 0.40306, step = 6472 (7.151 sec)
INFO:tensorflow:Saving checkpoints for 6534 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:Loss for final step: 0.320755.
INFO:tensor

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-9075
INFO:tensorflow:Saving checkpoints for 9076 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:loss = 0.315851, step = 9076
INFO:tensorflow:global_step/sec: 14.1815
INFO:tensorflow:loss = 0.323405, step = 9176 (7.053 sec)
INFO:tensorflow:global_step/sec: 14.2238
INFO:tensorflow:loss = 0.309933, step = 9276 (7.030 sec)
INFO:tensorflow:global_step/sec: 14.2255
INFO:tensorflow:loss = 0.349864, step = 9376 (7.030 sec)
INFO:tensorflow:Saving checkpoints for 9438 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:Loss for final step: 0.298381.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:23:39
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-9438
INFO:tensorflow:Finished evaluation at 2018-04-16-20:23:42
INFO:tensorflow:Saving dict for global step 9438: accuracy = 0.929744, cross_entropy_loss = 0.321331, global_step = 9438, loss = 0.613413
INFO:ten

INFO:tensorflow:global_step/sec: 13.7848
INFO:tensorflow:loss = 0.306988, step = 12080 (7.256 sec)
INFO:tensorflow:global_step/sec: 13.8434
INFO:tensorflow:loss = 0.296131, step = 12180 (7.223 sec)
INFO:tensorflow:global_step/sec: 13.8327
INFO:tensorflow:loss = 0.329329, step = 12280 (7.229 sec)
INFO:tensorflow:Saving checkpoints for 12342 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:Loss for final step: 0.287248.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:27:46
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-12342
INFO:tensorflow:Finished evaluation at 2018-04-16-20:27:49
INFO:tensorflow:Saving dict for global step 12342: accuracy = 0.928067, cross_entropy_loss = 0.354458, global_step = 12342, loss = 0.643197
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-12342
INFO:tensorflow:Saving checkpoints for 12343 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:loss = 0.298034, step = 1234

INFO:tensorflow:global_step/sec: 12.7529
INFO:tensorflow:loss = 0.31846, step = 15184 (7.841 sec)
INFO:tensorflow:Saving checkpoints for 15246 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:Loss for final step: 0.280241.
INFO:tensorflow:Starting evaluation at 2018-04-16-20:31:57
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-15246
INFO:tensorflow:Finished evaluation at 2018-04-16-20:32:00
INFO:tensorflow:Saving dict for global step 15246: accuracy = 0.927317, cross_entropy_loss = 0.385427, global_step = 15246, loss = 0.670854
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-15246
INFO:tensorflow:Saving checkpoints for 15247 into /tmp/tmpbzxc23gc/model.ckpt.
INFO:tensorflow:loss = 0.288547, step = 15247
INFO:tensorflow:global_step/sec: 12.7598
INFO:tensorflow:loss = 0.295001, step = 15347 (7.838 sec)
INFO:tensorflow:global_step/sec: 12.8464
INFO:tensorflow:loss = 0.286796, step = 15447 (7.784 sec)

INFO:tensorflow:Starting evaluation at 2018-04-16-20:36:10
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-18150
INFO:tensorflow:Finished evaluation at 2018-04-16-20:36:13
INFO:tensorflow:Saving dict for global step 18150: accuracy = 0.926522, cross_entropy_loss = 0.413076, global_step = 18150, loss = 0.695214


In [74]:
conf_score_u = list(conf_score_u_train) + list(conf_score_u_test)
conf_score_u = np.reshape(conf_score_u,newshape = (len(conf_score_u),))

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": U_sent, "ns":u_ns , "conf": conf_score_u},
                    batch_size=20, num_epochs=1, shuffle=False
                )

predictions = model1.predict(input_fn=predict_input_fn)

In [75]:
predictions = list(predictions)
pred_label_U = []

for i in range(len(predictions)):
    pred_label_U.append(predictions[i]['max'])

INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-18150


In [76]:
eval_metrics = model1.evaluate(input_fn=test_input_fn, name="test")  # replace with result of model.evaluate(...)


print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

INFO:tensorflow:Starting evaluation at 2018-04-16-20:40:12
INFO:tensorflow:Restoring parameters from /tmp/tmpbzxc23gc/model.ckpt-18150
INFO:tensorflow:Finished evaluation at 2018-04-16-20:40:15
INFO:tensorflow:Saving dict for global step 18150: accuracy = 0.926522, cross_entropy_loss = 0.413076, global_step = 18150, loss = 0.695214
Accuracy on test set: 92.65%


{'accuracy': 0.92652249,
 'cross_entropy_loss': 0.41307649,
 'global_step': 18150,
 'loss': 0.69521439}

In [77]:
pred_label_U = pd.DataFrame(data = pred_label_U, columns = ['pred_labels'])
pred_label_U.to_csv("predictions_12April.csv")

In [78]:
pred_label_U = pd.read_csv("predictions_12April.csv")

In [79]:
pred_label_U.count()

Unnamed: 0     113299
pred_labels    113299
dtype: int64

In [80]:
pred_label_U[pred_label_U == 2].count()

Unnamed: 0         1
pred_labels    27771
dtype: int64

In [81]:
pred_label_U[pred_label_U == 1].count()

Unnamed: 0         1
pred_labels    19037
dtype: int64

In [82]:
pred_label_U[pred_label_U == 0].count()

Unnamed: 0         1
pred_labels    66491
dtype: int64

In [83]:
pred_label_U_0 = []

for i in range(len(pred_label_U)):
    if pred_label_U.loc[i]['pred_labels'] == 0:
        pred_label_U_0.append(1)
    else:
        pred_label_U_0.append(0)

In [84]:
pred_label_U_1 = []

for i in range(len(pred_label_U)):
    if pred_label_U.loc[i]['pred_labels'] == 1:
        pred_label_U_1.append(1)
    else:
        pred_label_U_1.append(0)

In [85]:
pred_label_U_2 = []

for i in range(len(pred_label_U)):
    if pred_label_U.loc[i]['pred_labels'] == 2:
        pred_label_U_2.append(1)
    else:
        pred_label_U_2.append(0)

In [86]:
pred_label_U['label_0'] = pred_label_U_0

In [87]:
pred_label_U['label_1'] = pred_label_U_1

In [88]:
pred_label_U['label_2'] = pred_label_U_2

In [89]:
feature_size = U_sent.shape[1]


In [90]:
# Placeholders

input_x = tf.placeholder(tf.float32,shape=[None,feature_size])

In [91]:
num_classes = 3

In [92]:
input_y = tf.placeholder(tf.float32,shape=[None,num_classes])

In [93]:
hold_prob = tf.placeholder(tf.float32, name="hold_prob")

In [94]:
# training the model CNN and evluating it
# Now we define our model
from __future__ import print_function
from __future__ import division

import tensorflow as tf

def embedding_layer(ids_, V, embed_dim, init_scale=0.001):
    
    W_embed_ = tf.get_variable(name = 'W_embed', shape=[V,embed_dim],dtype=tf.float32
                               ,initializer=tf.random_uniform_initializer(minval= -init_scale
                                                                          ,maxval =init_scale),trainable=True)
    xs_ = tf.nn.embedding_lookup(W_embed_, ids_)
    return xs_

In [95]:
def fully_connected_layers(h0_, hidden_dims, activation=tf.tanh,
                           dropout_rate=0, is_training=False):
    h_ = h0_
    for i, hdim in enumerate(hidden_dims):
        
        h_ = tf.layers.dense(h_, hdim, activation=activation,    
                             kernel_initializer=tf.contrib.layers.xavier_initializer()
                             , bias_initializer=tf.zeros_initializer(), name = 'Hidden_%d'%i)
        
        if dropout_rate > 0:
            h_ = tf.nn.dropout(h_, keep_prob=dropout_rate)
            
        #if dropout_rate > 0:
            #h_ = tf.layers.dropout(inputs = h_, rate=dropout_rate, training = is_training)
        
    return h_

In [96]:
def softmax_output_layer(h_, labels_, num_classes):
    
    n = h_.get_shape().as_list()
    
    with tf.variable_scope("Logits"):
        W_out_ = tf.get_variable(name = 'W_out', shape=[n[1],num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        b_out_ = tf.get_variable(name = 'b_out', shape=[num_classes],dtype=tf.float32
                               ,initializer=tf.random_normal_initializer(),trainable=True)
        logits_ = tf.matmul(h_,W_out_) + b_out_
        
        

    # If no labels provided, don't try to compute loss.
    if labels_ is None:
        return None, logits_

    with tf.name_scope("Softmax"):
        loss_ = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_,logits=logits_)) 
        
    
    return loss_, logits_

In [97]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(self, sequence_length, num_classes, vocab_size,
        embedding_size, filter_heights, filter_widths,num_filters,channels_in,channels_out, init_scale,l2_reg_lambda):

        assert(len(filter_widths)==num_filters)
        assert(len(filter_heights)==num_filters)
        assert(len(channels_in)==num_filters)
        assert(len(channels_out)==num_filters)
        assert(channels_in[0]==1)
        
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        
        # Embedding layer
        with tf.name_scope("embedding_CNN"):
            W_embedding = tf.get_variable(name = 'W_embedding', shape=[vocab_size,embedding_size],dtype=tf.float32
                               ,initializer=tf.random_uniform_initializer(minval= -init_scale
                                                                          ,maxval =init_scale),trainable=True)
            self.xs = tf.nn.embedding_lookup(W_embedding, self.input_x)
            self.xs_expanded = tf.reshape(self.xs,[-1,sequence_length,embedding_size,1])
            
        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        i = 1
        x = self.xs_expanded
        for filter_h, filter_w, c_in, c_out in zip(filter_heights,filter_widths,channels_in,channels_out):
            with tf.name_scope("conv-maxpool-%s" % i):
                # Convolution Layer
                filter_shape = [filter_h, filter_w, c_in , c_out]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[c_out]), name="b")
                conv = tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME",name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(h,ksize=[1,embedding_size-filter_w+1, 1, 1],strides=[1, 1, 1, 1],padding="SAME",name="pool")
                x = pooled
                i = i + 1
                pooled_outputs.append(pooled)
        
        length = len(pooled_outputs)
        # Combine all the pooled features
        num_filters_total = c_out*sequence_length*embedding_size
        self.h_pool_flat = tf.reshape(pooled_outputs[length-1], [-1, num_filters_total])
        
        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
            
        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
            
        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy") 


In [98]:
# final padded sentences
X_sent_test, ns = utils.pad_np_array(X_ids, max_len=max_len2, pad_id=0)

In [99]:
Y_label = pd.DataFrame(data = Y_label, columns = ['labels'])
label_0 = []

for i in range(len(Y_label)):
    if Y_label.loc[i]['labels'] == 0:
        label_0.append(1)
    else:
        label_0.append(0)

In [100]:
label_1 = []

for i in range(len(Y_label)):
    if Y_label.loc[i]['labels'] == 1:
        label_1.append(1)
    else:
        label_1.append(0)

In [101]:
label_2 = []

for i in range(len(Y_label)):
    if Y_label.loc[i]['labels'] == 2:
        label_2.append(1)
    else:
        label_2.append(0)

In [102]:
Y_label['label_0'] = label_0
Y_label['label_1'] = label_1
Y_label['label_2'] = label_2

In [103]:
Y_label.iloc[0:600][['label_0','label_1','label_2']]         

,label_0,label_1,label_2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
5,1,0,0
6,1,0,0
7,1,0,0
8,1,0,0
9,1,0,0


In [104]:
batch_size = 100
num_epoch = 1
num_checkpoints = 5

with tf.Graph().as_default(), tf.Session() as sess:

    with sess.as_default():
        cnn = TextCNN(sequence_length=feature_size, num_classes=3, vocab_size=V,
        embedding_size=100, filter_heights=[5,5,5], filter_widths=[5,5,5],num_filters=3,channels_in=[1,3,3]
              ,channels_out=[3,3,3]
              ,init_scale=0.08, l2_reg_lambda=0.1)

        
        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)
        
        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)
        
        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=num_checkpoints)
        
        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 0.5
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)
            
        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
            
        from sklearn.model_selection import train_test_split 

        
        U_train,U_dev,y_train, y_dev = train_test_split(U_sent, pred_label_U[['label_0','label_1','label_2']], 
                                                        test_size=0.005, random_state=42)
        
        for j in range(num_epoch):
        
            t0 = time.time()
            total_batches = 0
            total_examples = 0
        
            for (bx, by) in utils.multi_batch_generator(batch_size, U_train, y_train):
                train_step(bx,by)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 50 == 0:
                    print("\nEvaluation:")
                    dev_step(U_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % 50 == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))
                    
        print("\nTest set accuracy:")
        dev_step(X_sent_test[0:600], Y_label.iloc[0:600][['label_0','label_1','label_2']], writer=dev_summary_writer)
        print("")

Writing to /home/ramya_girish/political_bias/political_bias/runs/1523911375

2018-04-16T20:43:02.009741: step 1, loss 1.39029, acc 0.4
2018-04-16T20:43:02.516896: step 2, loss 11.3052, acc 0.6
2018-04-16T20:43:03.018875: step 3, loss 7.95248, acc 0.57
2018-04-16T20:43:03.521031: step 4, loss 2.76381, acc 0.29
2018-04-16T20:43:04.023169: step 5, loss 3.49105, acc 0.3
2018-04-16T20:43:04.523649: step 6, loss 5.41642, acc 0.22
2018-04-16T20:43:05.024019: step 7, loss 2.98517, acc 0.16
2018-04-16T20:43:05.522870: step 8, loss 2.38735, acc 0.61
2018-04-16T20:43:06.021641: step 9, loss 2.71408, acc 0.64
2018-04-16T20:43:06.521300: step 10, loss 2.91001, acc 0.59
2018-04-16T20:43:07.020237: step 11, loss 2.57654, acc 0.54
2018-04-16T20:43:07.516527: step 12, loss 1.65551, acc 0.68
2018-04-16T20:43:08.013202: step 13, loss 1.74936, acc 0.59
2018-04-16T20:43:08.509969: step 14, loss 1.87808, acc 0.28
2018-04-16T20:43:09.006923: step 15, loss 1.67429, acc 0.23
2018-04-16T20:43:09.503159: step 16

2018-04-16T20:44:08.274798: step 130, loss 1.0154, acc 0.57
2018-04-16T20:44:08.763939: step 131, loss 0.938944, acc 0.63
2018-04-16T20:44:09.249735: step 132, loss 1.06762, acc 0.51
2018-04-16T20:44:09.736167: step 133, loss 0.996629, acc 0.58
2018-04-16T20:44:10.224549: step 134, loss 1.088, acc 0.49
2018-04-16T20:44:10.712383: step 135, loss 0.975097, acc 0.61
2018-04-16T20:44:11.200913: step 136, loss 0.908997, acc 0.67
2018-04-16T20:44:11.688850: step 137, loss 1.07322, acc 0.5
2018-04-16T20:44:12.176486: step 138, loss 0.939516, acc 0.63
2018-04-16T20:44:12.662890: step 139, loss 0.947468, acc 0.62
2018-04-16T20:44:13.149844: step 140, loss 0.944363, acc 0.62
2018-04-16T20:44:13.639495: step 141, loss 1.00068, acc 0.55
2018-04-16T20:44:14.128061: step 142, loss 1.0548, acc 0.54
2018-04-16T20:44:14.614825: step 143, loss 1.02315, acc 0.54
2018-04-16T20:44:15.102096: step 144, loss 0.940664, acc 0.61
2018-04-16T20:44:15.589785: step 145, loss 0.925949, acc 0.65
2018-04-16T20:44:16.

2018-04-16T20:45:12.267710: step 254, loss 1.00628, acc 0.54
2018-04-16T20:45:12.754739: step 255, loss 0.951986, acc 0.59
2018-04-16T20:45:13.242995: step 256, loss 0.96315, acc 0.59
2018-04-16T20:45:13.730943: step 257, loss 0.956107, acc 0.6
2018-04-16T20:45:14.219068: step 258, loss 1.00301, acc 0.55
2018-04-16T20:45:14.707338: step 259, loss 1.00894, acc 0.55
2018-04-16T20:45:15.195876: step 260, loss 0.991132, acc 0.56
2018-04-16T20:45:15.683920: step 261, loss 1.05364, acc 0.47
2018-04-16T20:45:16.170893: step 262, loss 0.940114, acc 0.6
2018-04-16T20:45:16.658866: step 263, loss 0.981324, acc 0.57
2018-04-16T20:45:17.146833: step 264, loss 0.931152, acc 0.64
2018-04-16T20:45:17.636265: step 265, loss 0.917951, acc 0.65
2018-04-16T20:45:18.124534: step 266, loss 1.0198, acc 0.53
2018-04-16T20:45:18.611843: step 267, loss 0.9814, acc 0.57
2018-04-16T20:45:19.100056: step 268, loss 0.908722, acc 0.64
2018-04-16T20:45:19.589058: step 269, loss 1.00357, acc 0.54
2018-04-16T20:45:20.

2018-04-16T20:46:16.645968: step 381, loss 0.880899, acc 0.66
2018-04-16T20:46:17.134908: step 382, loss 0.948663, acc 0.58
2018-04-16T20:46:17.622314: step 383, loss 0.976013, acc 0.56
2018-04-16T20:46:18.109634: step 384, loss 0.946924, acc 0.59
2018-04-16T20:46:18.597039: step 385, loss 0.993382, acc 0.52
2018-04-16T20:46:19.084236: step 386, loss 1.03216, acc 0.48
2018-04-16T20:46:19.571693: step 387, loss 0.935271, acc 0.61
2018-04-16T20:46:20.061165: step 388, loss 1.0159, acc 0.51
2018-04-16T20:46:20.549051: step 389, loss 0.949406, acc 0.62
2018-04-16T20:46:21.036793: step 390, loss 0.965707, acc 0.56
2018-04-16T20:46:21.522914: step 391, loss 0.967186, acc 0.53
2018-04-16T20:46:22.012239: step 392, loss 0.953824, acc 0.63
2018-04-16T20:46:22.499404: step 393, loss 0.923805, acc 0.59
2018-04-16T20:46:22.987638: step 394, loss 0.914874, acc 0.64
2018-04-16T20:46:23.475665: step 395, loss 0.857701, acc 0.68
2018-04-16T20:46:23.964223: step 396, loss 0.883201, acc 0.66
2018-04-16T

2018-04-16T20:47:21.097632: step 505, loss 0.589824, acc 0.78
2018-04-16T20:47:21.592325: step 506, loss 0.61951, acc 0.78
2018-04-16T20:47:22.087380: step 507, loss 0.687424, acc 0.75
2018-04-16T20:47:22.582522: step 508, loss 0.633855, acc 0.74
2018-04-16T20:47:23.078651: step 509, loss 0.599565, acc 0.78
2018-04-16T20:47:23.574532: step 510, loss 0.634571, acc 0.72
2018-04-16T20:47:24.071785: step 511, loss 0.623186, acc 0.76
2018-04-16T20:47:24.566325: step 512, loss 0.739762, acc 0.72
2018-04-16T20:47:25.062040: step 513, loss 0.662085, acc 0.72
2018-04-16T20:47:25.557745: step 514, loss 0.731041, acc 0.72
2018-04-16T20:47:26.053558: step 515, loss 0.657628, acc 0.78
2018-04-16T20:47:26.549624: step 516, loss 0.697166, acc 0.71
2018-04-16T20:47:27.043720: step 517, loss 0.685644, acc 0.74
2018-04-16T20:47:27.537640: step 518, loss 0.605948, acc 0.76
2018-04-16T20:47:28.031256: step 519, loss 0.625811, acc 0.77
2018-04-16T20:47:28.526487: step 520, loss 0.518421, acc 0.82
2018-04-1

2018-04-16T20:48:26.402192: step 632, loss 0.306608, acc 0.95
2018-04-16T20:48:26.899506: step 633, loss 0.379185, acc 0.87
2018-04-16T20:48:27.396951: step 634, loss 0.388155, acc 0.9
2018-04-16T20:48:27.891803: step 635, loss 0.372776, acc 0.89
2018-04-16T20:48:28.388032: step 636, loss 0.364359, acc 0.85
2018-04-16T20:48:28.883977: step 637, loss 0.401916, acc 0.86
2018-04-16T20:48:29.381391: step 638, loss 0.382723, acc 0.89
2018-04-16T20:48:29.878865: step 639, loss 0.655065, acc 0.78
2018-04-16T20:48:30.374679: step 640, loss 0.43935, acc 0.88
2018-04-16T20:48:30.869513: step 641, loss 0.458839, acc 0.88
2018-04-16T20:48:31.364769: step 642, loss 0.440415, acc 0.87
2018-04-16T20:48:31.859697: step 643, loss 0.405816, acc 0.9
2018-04-16T20:48:32.355510: step 644, loss 0.527001, acc 0.84
2018-04-16T20:48:32.851016: step 645, loss 0.390896, acc 0.92
2018-04-16T20:48:33.344086: step 646, loss 0.417749, acc 0.87
2018-04-16T20:48:33.838102: step 647, loss 0.39252, acc 0.92
2018-04-16T2

2018-04-16T20:49:31.346519: step 756, loss 0.242374, acc 0.95
2018-04-16T20:49:31.841498: step 757, loss 0.370989, acc 0.91
2018-04-16T20:49:32.336519: step 758, loss 0.209367, acc 0.93
2018-04-16T20:49:32.830580: step 759, loss 0.331918, acc 0.89
2018-04-16T20:49:33.326226: step 760, loss 0.248656, acc 0.94
2018-04-16T20:49:33.822571: step 761, loss 0.325892, acc 0.86
2018-04-16T20:49:34.318882: step 762, loss 0.269731, acc 0.92
2018-04-16T20:49:34.815887: step 763, loss 0.320498, acc 0.9
2018-04-16T20:49:35.310235: step 764, loss 0.351942, acc 0.93
2018-04-16T20:49:35.805304: step 765, loss 0.33705, acc 0.92
2018-04-16T20:49:36.300761: step 766, loss 0.284868, acc 0.93
2018-04-16T20:49:36.795530: step 767, loss 0.256533, acc 0.92
2018-04-16T20:49:37.291787: step 768, loss 0.357518, acc 0.9
2018-04-16T20:49:37.786033: step 769, loss 0.344065, acc 0.92
2018-04-16T20:49:38.280958: step 770, loss 0.218964, acc 0.95
2018-04-16T20:49:38.775808: step 771, loss 0.279708, acc 0.93
2018-04-16T

2018-04-16T20:50:36.628833: step 883, loss 0.20786, acc 0.94
2018-04-16T20:50:37.123076: step 884, loss 0.372644, acc 0.89
2018-04-16T20:50:37.618380: step 885, loss 0.184726, acc 0.96
2018-04-16T20:50:38.114535: step 886, loss 0.231675, acc 0.93
2018-04-16T20:50:38.610888: step 887, loss 0.191556, acc 0.97
2018-04-16T20:50:39.106691: step 888, loss 0.188507, acc 0.96
2018-04-16T20:50:39.599745: step 889, loss 0.295368, acc 0.91
2018-04-16T20:50:40.095012: step 890, loss 0.281768, acc 0.93
2018-04-16T20:50:40.589275: step 891, loss 0.291575, acc 0.9
2018-04-16T20:50:41.083042: step 892, loss 0.465324, acc 0.89
2018-04-16T20:50:41.577040: step 893, loss 0.341855, acc 0.93
2018-04-16T20:50:42.069824: step 894, loss 0.312552, acc 0.9
2018-04-16T20:50:42.563264: step 895, loss 0.235998, acc 0.93
2018-04-16T20:50:43.055110: step 896, loss 0.286889, acc 0.91
2018-04-16T20:50:43.547638: step 897, loss 0.336652, acc 0.89
2018-04-16T20:50:44.040494: step 898, loss 0.324018, acc 0.9
2018-04-16T2

2018-04-16T20:51:41.327691: step 1007, loss 0.225469, acc 0.93
2018-04-16T20:51:41.820842: step 1008, loss 0.181918, acc 0.96
2018-04-16T20:51:42.313930: step 1009, loss 0.552669, acc 0.88
2018-04-16T20:51:42.808580: step 1010, loss 0.283248, acc 0.92
2018-04-16T20:51:43.301203: step 1011, loss 0.155572, acc 0.98
2018-04-16T20:51:43.793433: step 1012, loss 0.279133, acc 0.92
2018-04-16T20:51:44.286249: step 1013, loss 0.300521, acc 0.91
2018-04-16T20:51:44.779676: step 1014, loss 0.216778, acc 0.94
2018-04-16T20:51:45.271644: step 1015, loss 0.267163, acc 0.94
2018-04-16T20:51:45.763962: step 1016, loss 0.169658, acc 0.96
2018-04-16T20:51:46.255966: step 1017, loss 0.234004, acc 0.95
2018-04-16T20:51:46.746834: step 1018, loss 0.186452, acc 0.97
2018-04-16T20:51:47.238125: step 1019, loss 0.273036, acc 0.95
2018-04-16T20:51:47.729753: step 1020, loss 0.293845, acc 0.9
2018-04-16T20:51:48.219291: step 1021, loss 0.231793, acc 0.95
2018-04-16T20:51:48.710420: step 1022, loss 0.256665, ac